In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
from datetime import datetime 

# Load API key from .env file
load_dotenv()
coinapi_key = os.getenv('api_key')

exchange_id = "DERIBIT"

# CoinAPI endpoint for symbols metadata
url = f"https://rest.coinapi.io/v1/options/{exchange_id}/current"

#payload = {}
headers = {
  'Accept': 'text/plain',
  'X-CoinAPI-Key': coinapi_key
}

#response = requests.request("GET", url, headers=headers, data=payload)
response = requests.request("GET", url, headers=headers)

#print(response.text)

# Check for successful response
if response.status_code == 200:
    data = response.json()
    
    # Filter for BTC/USD call options
    filtered_calls = []
    for option in data:
        if option.get("asset_id_base") == "BTC" and option.get("asset_id_quote") == "USD":
            underlying_price = option.get("underlying_price")
            time_expiration = option.get("time_expiration")

            for strike in option.get("strikes", []):
                if "call" in strike:
                    call_data = {
                        "symbol_id": strike["call"]["symbol_id"],
                        "time_exchange": strike["call"]["time_exchange"],
                        "time_coinapi": strike["call"]["time_coinapi"],
                        "strike_price": strike["strike_price"],
                        "ask_price": strike["call"]["ask_price"],
                        "ask_size": strike["call"]["ask_size"],
                        "bid_price": strike["call"]["bid_price"],
                        "bid_size": strike["call"]["bid_size"],
                        "underlying_price": underlying_price,
                        "time_expiration": time_expiration
                    }
                    filtered_calls.append(call_data)

    # Convert to DataFrame for better visualization
    df_calls = pd.DataFrame(filtered_calls)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)
    print(df_calls)

else:
    print(f"Error: {response.status_code}, {response.text}")


                               symbol_id                 time_exchange                  time_coinapi  strike_price  ask_price  ask_size  bid_price  bid_size  underlying_price               time_expiration
0    DERIBIT_OPT_BTC_USD_250425_104000_C  2025-02-04T08:59:49.0661066Z  2025-02-04T08:59:49.0661250Z      104000.0     0.0910       2.2     0.0875       2.2      98479.653102  2025-04-25T08:00:00.0000000Z
1    DERIBIT_OPT_BTC_USD_250425_175000_C  2025-02-04T08:59:50.0697020Z  2025-02-04T08:59:50.0697422Z      175000.0     0.0070       4.6     0.0055       1.0      98479.653102  2025-04-25T08:00:00.0000000Z
2    DERIBIT_OPT_BTC_USD_250425_118000_C  2025-02-04T09:00:01.3933261Z  2025-02-04T09:00:01.3933446Z      118000.0     0.0500       6.0     0.0460       6.0      98479.653102  2025-04-25T08:00:00.0000000Z
3     DERIBIT_OPT_BTC_USD_250425_92000_C  2025-02-04T08:59:42.7111738Z  2025-02-04T08:59:42.7112037Z       92000.0     0.1495       3.2     0.1440       3.0      98479.653102  2025

# Data Preparation

### Calculate 
### Market Price using ((ask price + bid price) / 2)
### Maturity(Years) using time_expiration and time_exchange
### Risk-Free Rate is gotten from OpenBB


### Maturity Years

In [3]:
df_calls = pd.DataFrame(filtered_calls)

#convert time columns to datetime
df_calls['time_exchange'] = pd.to_datetime(df_calls['time_exchange'])
df_calls['time_expiration'] = pd.to_datetime(df_calls['time_expiration'])

#Calculate the maturity in years
df_calls['maturity_years'] = (df_calls['time_expiration'] - df_calls['time_exchange']).dt.total_seconds() / (365.25 * 24 * 3600)
df_calls['maturity_years']

0      0.218914
1      0.218914
2      0.218914
3      0.218915
4      0.218914
         ...   
413    0.391399
414    0.391399
415    0.391399
416    0.391399
417    0.391399
Name: maturity_years, Length: 418, dtype: float64

### Market Price

In [4]:
df_calls['market_price'] = (df_calls['ask_price'] + df_calls['bid_price']) / 2
df_calls['market_price']

0      0.08925
1      0.00625
2      0.04800
3      0.14675
4      0.05450
        ...   
413    0.01900
414    0.44500
415    0.07425
416    0.07350
417    0.01525
Name: market_price, Length: 418, dtype: float64